In [1]:
from pyomo.environ import ConcreteModel, SolverFactory
                           
from idaes.core import FlowsheetBlock
                           
from idaes.generic_models.properties.core.generic.generic_property import (
        GenericParameterBlock)
                           
from R32_pf6_k import configuration

from idaes.generic_models.unit_models import Flash

from idaes.core.util.model_statistics import degrees_of_freedom

import idaes.logger as idaeslog

# Import plotting functions
import matplotlib.pyplot as plt

# Import numpy library 
import numpy as np

In [2]:
m = ConcreteModel()

m.fs = FlowsheetBlock(default={"dynamic": False})

m.fs.properties = GenericParameterBlock(default=configuration)

m.fs.F101 = Flash(default={"property_package": m.fs.properties,
                           "has_heat_transfer": False,
                           "has_pressure_change": False})

2021-07-19 14:06:20 [INFO] idaes.generic_models.properties.core.generic.generic_property: DEPRECATED - definiton of generic property packages is moving to using static classes instead of modules. Please refer to the IDAES documentation.


In [3]:
DOF_initial = degrees_of_freedom(m)
print("The initial DOF is {0}".format(DOF_initial))

The initial DOF is 5


In [4]:
m.fs.F101.inlet.flow_mol.fix(1)
m.fs.F101.inlet.temperature.fix(298)
m.fs.F101.inlet.pressure.fix(100000)

m.fs.F101.inlet.mole_frac_comp[0, "R32"].fix(0.4)
m.fs.F101.inlet.mole_frac_comp[0, "bmimPF6"].fix(0.6)
# m.fs.F101.vap_outlet.pressure.fix(399800)

# m.fs.F101.vap_outlet.temperature.fix(286)

In [5]:
degrees_of_freedom(m.fs.F101)

0

In [6]:
# Initialize the flash unit
m.fs.F101.initialize()

2021-07-19 14:06:20 [INFO] idaes.init.fs.F101.control_volume.properties_in: Starting initialization
2021-07-19 14:06:21 [INFO] idaes.init.fs.F101.control_volume.properties_in: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-07-19 14:06:21 [INFO] idaes.init.fs.F101.control_volume.properties_in: Equilibrium temperature initialization completed.
2021-07-19 14:06:21 [INFO] idaes.init.fs.F101.control_volume.properties_in: Phase equilibrium initialization: optimal - Optimal Solution Found.
2021-07-19 14:06:21 [INFO] idaes.init.fs.F101.control_volume.properties_in: Property initialization: optimal - Optimal Solution Found.
2021-07-19 14:06:21 [INFO] idaes.init.fs.F101.control_volume.properties_out: Starting initialization
2021-07-19 14:06:21 [INFO] idaes.init.fs.F101.control_volume.properties_out: Dew and bubble point initialization: optimal - Optimal Solution Found.
2021-07-19 14:06:21 [INFO] idaes.init.fs.F101.control_volume.properties_out: Equilibrium temperatur

In [7]:
solver = SolverFactory('ipopt')
solver.options = {'tol': 1e-6}

In [8]:
status = solver.solve(m, tee = False)

In [9]:
m.fs.F101.report()


Unit : fs.F101                                                             Time: 0.0
------------------------------------------------------------------------------------
    Unit Performance


------------------------------------------------------------------------------------
    Stream Table
                              Inlet    Vapor Outlet  Liquid Outlet
    flow_mol                   1.0000      0.36136       0.63864  
    mole_frac_comp bmimPF6    0.60000   1.0000e-08       0.93950  
    mole_frac_comp R32        0.40000       1.0000      0.060501  
    temperature                298.00       298.00        298.00  
    pressure               1.0000e+05   1.0000e+05    1.0000e+05  


In [10]:
m.display()

Model unknown

  Variables:
    None

  Objectives:
    None

  Constraints:
    None

  Blocks:
    Block fs
    
      Variables:
        None
    
      Objectives:
        None
    
      Constraints:
        None
    
      Blocks:
        Block fs.properties
        
          Variables:
            PR_kappa_A : PRA binary interaction parameters
                Size=4, Index=fs.properties.PR_kappa_A_index
                Key                    : Lower : Value : Upper : Fixed : Stale : Domain
                        ('R32', 'R32') :  None :   0.0 :  None :  True :  True :  Reals
                    ('R32', 'bmimPF6') :  None :   0.0 :  None :  True :  True :  Reals
                    ('bmimPF6', 'R32') :  None :   0.0 :  None :  True :  True :  Reals
                ('bmimPF6', 'bmimPF6') :  None :   0.0 :  None :  True :  True :  Reals
            PR_kappa_B : PRB binary interaction parameters
                Size=4, Index=fs.properties.PR_kappa_B_index
                Key      

In [11]:
# m.pprint()